# One-dimensional shear flows in Cartesian and cylindrical geometries
This notebook contains the validation tests for problems of modal and non-modal linear stability of 1D shear flows, such as plane Poiseuille flow, Couette flow, and pipe Poiseuille flow. In each case, the reference results and the present results are displayed side-by-side in `pandas` tables.

In [8]:
import numpy as np
import scipy.linalg as la
import pandas
import trispectral as ts
from operator_matrices import *

## Reference eigenvalues
Present eigenvalues are taken from the Schmid & Henningson (2001) book.

In [2]:
SCHMID_HENNINGSON_TABLES = {
    "poiseuille": { # Re = 2000, (α, β) = (1, 0), (0.5, 1), (0.25, 2)
        (1., 0.): np.array(
            [
                0.98418861 - 0.01631139j,
                0.31210030 - 0.01979866j,
                0.95256584 - 0.04793417j,
                0.42418427 - 0.07671992j,
                0.92078667 - 0.07804706j,
                0.92091806 - 0.07820060j,
                0.92094306 - 0.07955694j,
                0.88932028 - 0.11117972j,
                0.24936056 - 0.13725811j,
                0.24936056 - 0.13725811j,
                0.85717055 - 0.13990151j,
                0.85758968 - 0.14031674j,
                0.85769752 - 0.14280249j,
                0.82607494 - 0.17442537j,
                0.79399812 - 0.20190508j,
                0.79413424 - 0.20232063j,
                0.79445264 - 0.20605114j,
                0.63912513 - 0.22134137j,
                0.53442105 - 0.22356175j,
                0.42863639 - 0.22466515j,
            ]
        ),
        (0.5, 1.): np.array(
            [
                0.97763932 - 0.02361068j,
                0.37226932 - 0.03737398j,
                0.93291797 - 0.06833204j,
                0.49935557 - 0.09920592j,
                0.88770220 - 0.10945538j,
                0.88808805 - 0.10962449j,
                0.88819669 - 0.11305351j,
                0.84347472 - 0.15777756j,
                0.31232252 - 0.16986946j,
                0.31232252 - 0.16986946j,
                0.79534673 - 0.19331077j,
                0.79830962 - 0.19657914j,
                0.79871747 - 0.20251584j,
                0.75360282 - 0.24701616j,
                0.72648153 - 0.26096201j,
                0.64779065 - 0.26971348j,
                0.53467243 - 0.27140532j,
                0.53470480 - 0.27146759j,
                0.70474692 - 0.29872825j,
                0.43320720 - 0.30659209j,
            ]
        ),
        (0.25, 2.): np.array(
            [
                0.96837728 - 0.03974775j,
                0.56329537 - 0.08548514j,
                0.90513428 - 0.10299426j,
                0.83796079 - 0.14010066j,
                0.84492959 - 0.14965217j,
                0.58717755 - 0.15289251j,
                0.84190728 - 0.16629312j,
                0.39061440 - 0.21452290j,
                0.39061296 - 0.21452788j,
                0.72738833 - 0.22753394j,
                0.77824142 - 0.23018702j,
                0.73333862 - 0.28917720j,
                0.70571029 - 0.29246161j,
                0.65305016 - 0.31673071j,
                0.65688684 - 0.35866635j,
                0.63548529 - 0.36625533j,
                0.62259513 - 0.41560943j,
                0.64672215 - 0.44988239j,
                0.67269250 - 0.46026968j,
                0.66265886 - 0.56491518j,
            ]
        ),
    },
    "couette": { # Re = 800, (α, β) = (1, 0), (0.5, 1), (0.25, 2)
        (1., 0.): np.array(
            [
                +0.78187852 - 0.12718249j,
                -0.78187852 - 0.12718249j,
                +0.57647380 - 0.12952206j,
                -0.57647380 - 0.12952206j,
                +0.61863618 - 0.22143050j,
                -0.61863618 - 0.22143050j,
                +0.33837303 - 0.28699889j,
                -0.33837303 - 0.28699889j,
                +0.48498831 - 0.29859214j,
                -0.48498831 - 0.29859214j,
                +0.65474385 - 0.31845690j,
                -0.65474385 - 0.31845690j,
                +0.36686964 - 0.36678798j,
                -0.36686964 - 0.36678798j,
                +0.13853225 - 0.41451617j,
                -0.13853225 - 0.41451617j,
                +0.25889370 - 0.42912792j,
                -0.25889370 - 0.42912792j,
                +0.39155287 - 0.44983177j,
                -0.39155287 - 0.44983177j,
                +0.15827968 - 0.48721835j,
                -0.15827968 - 0.48721835j,
                +0.         - 0.51543904j,
                +0.06311251 - 0.54178178j,
                -0.06311251 - 0.54178178j,
                +0.17496627 - 0.56239996j,
                -0.17496627 - 0.56239996j,
                +0.         - 0.59624095j,
                +0.         - 0.64277951j,
                +0.         - 0.69476878j,
                +0.         - 0.69585432j,
                +0.         - 0.80564720j,
                +0.         - 0.80692602j,
            ]
        ),
        (0.5, 1.): np.array(
            [
                +0.72518416 - 0.16179000j,
                -0.72518416 - 0.16179000j,
                +0.47711120 - 0.17057387j,
                -0.47711120 - 0.17057387j,
                +0.51951170 - 0.28053505j,
                -0.51951170 - 0.28053505j,
                +0.18033226 - 0.36896022j,
                -0.18033226 - 0.36896022j,
                +0.35112593 - 0.37775262j,
                -0.35112593 - 0.37775262j,
                +0.55316723 - 0.39709328j,
                -0.55316723 - 0.39709328j,
                +0.20230357 - 0.46367578j,
                -0.20230357 - 0.46367578j,
                +0.         - 0.52654272j,
                +0.06726756 - 0.54144268j,
                -0.06726756 - 0.54144268j,
                +0.21826018 - 0.56176351j,
                -0.21826018 - 0.56176351j,
                +0.         - 0.65186131j,
                +0.         - 0.68690207j,
                +0.         - 0.80374929j,
                +0.         - 0.80563552j,
                +0.         - 0.96841110j,
                +0.         - 0.97104292j,
                +0.         - 1.14407304j,
                +0.         - 1.14689443j,
                +0.         - 1.32986473j,
                +0.         - 1.33355240j,
            ]
        ),
        (0.25, 2.): np.array(
            [
                +0.65375374 - 0.22021787j,
                -0.65375374 - 0.22021787j,
                +0.39990294 - 0.25329463j,
                -0.39990294 - 0.25329463j,
                +0.39462265 - 0.36982719j,
                -0.39462265 - 0.36982719j,
                +0.37283853 - 0.48343132j,
                -0.37283853 - 0.48343132j,
                +0.         - 0.48561430j,
                +0.18249973 - 0.49241970j,
                -0.18249973 - 0.49241970j,
                +0.         - 0.56306665j,
                +0.         - 0.57627651j,
                +0.         - 0.71401555j,
                +0.         - 0.72239155j,
                +0.         - 0.92956625j,
                +0.         - 0.94254293j,
                +0.         - 1.17658449j,
                +0.         - 1.19022327j,
                +0.         - 1.44426340j,
                +0.         - 1.45959937j,
                +0.         - 1.73554371j,
                +0.         - 1.75143991j,
                +0.         - 2.04876750j,
                +0.         - 2.06630407j,
            ]
        ),
    },
    "pipe": { # Re = 2000, α = 0.25, n = 2
        (0.25, 2): np.array(
            [
                0.72551688 - 0.14895301j,
                0.37381075 - 0.17973957j,
                0.51310797 - 0.19545604j,
                0.85236191 - 0.27489963j,
                0.59574165 - 0.29082257j,
                0.60783255 - 0.40189549j,
                0.72720893 - 0.41549788j,
                0.64896591 - 0.50429278j,
                0.66139917 - 0.61313884j,
                0.66664316 - 0.73870170j,
                0.66017652 - 0.86358022j,
                0.66968104 - 1.00995818j,
                0.65952947 - 1.15557527j,
                0.67077599 - 1.31762599j,
                0.65971050 - 1.48607071j,
                0.67093631 - 1.66406005j,
                0.66029737 - 1.85486324j,
                0.67069161 - 2.04976452j,
                0.66099154 - 2.26220681j,
                0.67030895 - 2.47485248j,
            ]
        ),
    },
}

## Plane Poiseuille flow and Couette flow

To compare with the results of Schmid & Henningson, the flow parameters must be set to the following values:

* Plane Poiseuille flow:

    Flow type: `flow = "poiseuille"`

    Reynolds number: `Re = 2000.`
    
    Combinations of $\alpha$ and $\beta$: `alpha, beta = 1., 0.`, `alpha, beta = 0.5, 1.`, `alpha, beta = 0.25, 2.`

* Couette flow:

    Flow type: `flow = "couette"`

    Reynolds number: `Re = 800.`
    
    Combinations of $\alpha$ and $\beta$: `alpha, beta = 1., 0.`, `alpha, beta = 0.5, 1.`, `alpha, beta = 0.25, 2.`

In [3]:
nx = 81
flow = "poiseuille"
Re = 2000.
alpha = 0.25
beta = 2.

In [5]:
x = ts.Grid.from_bounds([-1., 1., nx], discs=["chebyshev"])

We compute the eigenvalues using the Orr-Sommerfeld-Squire formulation and the method of algebraic reduction.

In [7]:
lmat, rmat = oss_cartesian_matrices(
    flow, x, Re=Re, alpha=alpha, beta=beta
)

ω_oss = la.eigvals(la.inv(rmat) @ lmat)

ω_oss = ω_oss[ω_oss.imag.argsort()[::-1]]
ω_oss = ω_oss[~np.isclose(ω_oss, -100j)]

In [11]:
nval = 20 # number of eigenvalues to show in a table

table = pandas.DataFrame({})

if (
    ((flow in "poiseuille" and Re == 2000.) or (flow in "couette" and Re == 800.))
    and (alpha, beta) in SCHMID_HENNINGSON_TABLES[flow].keys()
):
    ref_vals = SCHMID_HENNINGSON_TABLES[flow][(alpha, beta)] * alpha
    ref_col = pandas.DataFrame(
        {"Schmid & Henningson (2001)": ref_vals}
    )
    table = pandas.concat([table, ref_col], axis=1)

    nval = SCHMID_HENNINGSON_TABLES[flow][(alpha, beta)].size

table = pandas.concat(
    [
        table,
        pandas.DataFrame({"OSS": ω_oss[:nval], "AR": ω_oss[:nval]}),
    ],
    axis=1,
)

s = table.style.format(
    precision=8, decimal="."
).relabel_index(
    [str(i) for i in np.arange(1, nval + 1)], axis=0
).set_caption(f"Eigenvalues for {flow[0].upper() + flow[1:]} flow")

headers = {
    "selector": "th",
    "props": "background-color: #000066; color: white;",
}
data = {
    "selector": "td",
    "props": "text-align: center; font-style: italic;",
}
s = s.set_table_styles([headers, data])

display(s)

,Schmid & Henningson (2001),OSS,AR
1,0.24209432-0.00993694j,0.24209432-0.00993694j,0.24209432-0.00993694j
2,0.14082384-0.02137129j,0.14082384-0.02137129j,0.14082384-0.02137129j
3,0.22628357-0.02574857j,0.22628357-0.02574856j,0.22628357-0.02574856j
4,0.20949020-0.03502516j,0.20949020-0.03502517j,0.20949020-0.03502517j
5,0.21123240-0.03741304j,0.21123240-0.03741304j,0.21123240-0.03741304j
6,0.14679439-0.03822313j,0.14679439-0.03822313j,0.14679439-0.03822313j
7,0.21047682-0.04157328j,0.21047682-0.04157328j,0.21047682-0.04157328j
8,0.09765360-0.05363072j,0.09765360-0.05363073j,0.09765360-0.05363073j
9,0.09765324-0.05363197j,0.09765324-0.05363197j,0.09765324-0.05363197j
10,0.18184708-0.05688348j,0.18184708-0.05688349j,0.18184708-0.05688349j
